In [ ]:
import pandas as pd
import re
import datetime as dt
import numpy as np

In [ ]:
# read 2nd sheet of an excel file
ddid = pd.read_excel(r"/Users/brentcleland/NSS/Projects/ddid-rasberry-pi/data/TEIS Exit Data for NSS.xlsx", sheet_name =3)

In [ ]:
ddid[['Child ID', 'DOB', 'Initial IFSP Date', 'Exit Date', 'Exit Reason', 'Exit Note']]

In [ ]:
ddid_filtered = ddid[ddid['Exit Note'].notnull()]
ddid_filtered.head()

In [ ]:
#ddid_filtered['Exit Note']

In [ ]:
#ddid_filtered.loc[ddid_filtered['Exit Note'].str.contains(r"(\d+/\d+/\d+)", case=False)]


In [ ]:
#search.to_csv('new_filter.csv')

In [ ]:
search = ddid_filtered.loc[ddid_filtered['Exit Note'].str.contains('(Reopened:)\s\d{2}/\d{2}/\d{4}\s(Exit Date:)\s\d{2}/\d{2}/\d{4}')]
search

In [ ]:
search['Exit Note'].str.extract(r'(Reopened:\d{2}/\d{2}/\d{4})')
search.head(2)

Findall messed up the str function below we used extract

In [ ]:
#search['Exit Note'].str.findall(r'Reopened:\d{2}/\d{2}/\d{4}')
#search.head(2)

In [ ]:
dates = search['Exit Note'].str.extract(r'(Reopened: \d{2}/\d{2}/\d{4}) (Exit Date: \d{2}/\d{2}/\d{4})')
dates

In [ ]:
#dates = search['Exit Note'].str.findall(r'(Reopened: \d{2}/\d{2}/\d{4}) (Exit Date: \d{2}/\d{2}/\d{4})')
#dates

In [ ]:
reopen_adjusted = search['Exit Note'].str.extract(r'(Reopened: \d{2}/\d{2}/\d{4})')
reopen_adjusted

In [ ]:
#reopen_adjusted = search['Exit Note'].str.findall(r'(Reopened: \d{2}/\d{2}/\d{4})')
#reopen_adjusted

In [ ]:
exit_adjusted  = search['Exit Note'].str.extract(r'(Exit Date: \d{2}/\d{2}/\d{4})')
exit_adjusted

In [ ]:
#exit_adjusted  =search['Exit Note'].str.findall(r'(Exit Date: \d{2}/\d{2}/\d{4})')
#exit_adjusted

In [ ]:
#exit_adjusted1=search['Exit Note'].str.findall(r'\d{2}/\d{2}/\d{4}')
#exit_adjusted1

In [ ]:
search['reopen_date'] = reopen_adjusted
search.head()

In [ ]:
search['exit_date_adjusted'] = exit_adjusted
search.head()

In [ ]:
#search['exit_date_adjusted'] = search['exit_date_adjusted'].str.extract('(\d+.\d+.\d+)')
#search

In [ ]:
exit_val = search['exit_date_adjusted'].str.extract('(\d+.\d+.\d+)')
exit_val

In [ ]:
#exit_val = search['exit_date_adjusted'].astype(str).str.extract('(\d+.\d+.\d+\ Exit Reason: 618)')
#exit_val.to_csv('x.csv')

In [ ]:
reopen_val = search['reopen_date'].astype(str).str.extract('(\d+.\d+.\d+)')


In [ ]:
search['reopen_date'] = reopen_val
search

In [ ]:
search['exit_date_adjusted'] = exit_val
search.head()

In [ ]:
display(search.dtypes)

In [ ]:
#pd.to_datetime(df['your_time_column'])
pd.to_datetime(search['reopen_date'])

In [ ]:
#pd.to_datetime(df['your_time_column'])
pd.to_datetime(search['exit_date_adjusted'])

In [ ]:
search['reopen_date'] = pd.to_datetime(search['reopen_date'])


In [ ]:
search['exit_date_adjusted'] = pd.to_datetime(search['exit_date_adjusted'])

#df['date_column'] = pd.to_datetime(df['datetime_column']).dt.date


In [ ]:
search

In [ ]:
search['Adjusted_Count'] = search['reopen_date'] - search['exit_date_adjusted']
search.head()

In [ ]:
for index, row in search.iterrows():
    if ((row['exit_date_adjusted'] > row['Initial IFSP Date']) and
        (row['exit_date_adjusted'] < row['Exit Date']) and
        (row['reopen_date'] > row['Initial IFSP Date']) and
        (row['reopen_date'] < row['Exit Date'])):
        diff_days = (row['reopen_date'] - row['exit_date_adjusted']).days
        search.at[index, 'diff_days'] = diff_days
    else:
        search.at[index, 'diff_days'] = np.nan

In [ ]:
search.head()

In [ ]:
search_filtered = search[search['diff_days'].notnull()]
search.head()

In [ ]:
num_rows = search_filtered.shape[0]
num_rows

In [ ]:
search_filtered['Exit Note'].str.count('(Exit Date: \d+.\d+.\d+)')
search_filtered['Exit Note'].str.count('(Reopened: \d+.\d+.\d+)')


In [ ]:
search['Exit_Count'] = search["Exit Note"].str.count('(Exit Date: \d+.\d+.\d+)')
search['Reopen_Count'] = search["Exit Note"].str.count('(Reopened: \d+.\d+.\d+)')
search.head()
#ddid_search['Exit Note'].str.count('(Reopened: \d+.\d+.\d+)')


In [ ]:
search_edit = search
search_edit.head(1)

In [ ]:
search_edit['summed_counts'] = search_edit['Exit_Count'] + search_edit['Reopen_Count']
search_edit

In [ ]:
newexit = pd.to_datetime(search_edit['Exit Note'].str.extractall('Exit Date: (\d+.\d+.\d+)')[0]).unstack()
newexit.columns = ['exit_' + str(col + 1) for col in newexit.columns]
newexit

In [ ]:
newenter = pd.to_datetime(search_edit['Exit Note'].str.extractall('Reopened: (\d+.\d+.\d+)')[0]).unstack()
newenter.columns = ['enter_' + str(col + 1) for col in newenter.columns]
newenter

In [ ]:
rentry_df = pd.merge(newexit, newenter, left_index=True, right_index=True)
#rentry_df.to_csv('re-enter.csv')

In [ ]:
search_all = pd.merge(search_edit,  rentry_df,  left_index=True, right_index=True)

In [ ]:
#search_all.to_csv('search_all.csv')

In [ ]:
search_all.columns

In [ ]:
for index, row in search_all.iterrows():
    new_diff = 0
    for num in range(1,7):
        if ((row[f'exit_{num}'] != row[f'enter_{num}']) and
            (row[f'exit_{num}'] != np.nan)  and
            (row[f'enter_{num}'] != np.nan) and
            (row[f'exit_{num}'] > row['Initial IFSP Date']) and
            (row[f'exit_{num}'] < row['Exit Date']) and
            (row[f'enter_{num}'] > row['Initial IFSP Date']) and
            (row[f'enter_{num}'] < row['Exit Date'])):
            mult_diff_days = (row[f'enter_{num}'] - row[f'exit_{num}']).days
            new_diff += mult_diff_days
            search_all.at[index, 'diff_days_new'] = new_diff
        else:
            pass
#print(search_all.head(15))
#search_all.to_csv('search_all.csv')

In [ ]:
#import pandas as pd

#df = {"A1":[15,25], "B1": [30, 40], "A2":[50,60], "B2": [70, 80]}
#df = pd.DataFrame(df)
#first_cols = ["A1", "B1"]
#second_cols = ["A2", "B2"]

#sub_cols = ["A_sub","B_sub"]

#df[sub_cols] = df[first_cols] - df[second_cols].values


In [ ]:

search_all['new_calc_date'] = search_all['Days from I-IFSP to Exit'] + search_all['diff_days_new']
search_all

In [ ]:

search_all['Exit Note'].str.extract('(Exit Reason: )').unstack()

In [ ]:
#newexit = pd.to_datetime(search_edit['Exit Note'].str.extractall('Exit Reason: (\d+.\d+.\d+)')[0]).unstack()
#newexit.columns = ['exit_' + str(col + 1) for col in newexit.columns]
newexit

In [ ]:
c = ddid['Exit Note'].str.partition('(Exit Reason:)')[0]
c

pandas.pivot_table(data, values=None, index=None, columns=None, aggfunc='mean', 
                   fill_value=None, margins=False, dropna=True, margins_name='All', 
                   observed=False, sort=True)[source]

In [ ]:
pd.pivot_table(search_all, values='new_calc_date', index=['POE'], columns=['Exit Reason'], aggfunc=np.sum)

In [ ]:
# read 2nd sheet of an excel file
eco = pd.read_excel(r"/Users/brentcleland/NSS/Projects/ddid-rasberry-pi/data/TEIS Exit Data for NSS.xlsx", sheet_name =2)

In [ ]:
#print(eco.head())

In [ ]:
pivot = pd.merge(search_all, eco, how='left', left_on='Child ID', right_on ='CHILD_ID')
#rentry_df.to_csv('re-enter.csv')

In [ ]:
pivot


In [ ]:
oc1 = pd.pivot_table(pivot, values='new_calc_date', index= ['POE'], columns=('OC1 - b' ),
                     aggfunc=np.mean, margins=sum)
oc1

In [ ]:
oc2 = pd.pivot_table(pivot, values='new_calc_date', index= ['POE'], columns=('OC2 - b' ),
                     aggfunc=np.median, margins=sum)
oc2

In [ ]:
oc3 = pd.pivot_table(pivot, values='new_calc_date', index= ['POE'], columns=('OC3 - b',
                                                                            ) ,aggfunc=np.mean, margins=sum)
oc3

In [ ]:
oc1 = pd.pivot_table(pivot, values='new_calc_date', index= ['POE'], columns=('OC1 - d' ),aggfunc=np.mean, margins=sum)
oc2 = pd.pivot_table(pivot, values='new_calc_date', index= ['POE'], columns=('OC2 - d' ),aggfunc=np.mean, margins=sum)
oc3 = pd.pivot_table(pivot, values='new_calc_date', index= ['POE'], columns=('OC3 - d' ),aggfunc=np.mean, margins=sum)
oc1.to_csv('oc1d.csv')
oc2.to_csv('oc2d.csv')
oc3.to_csv('oc3d.csv')